# ASR-Stage

In this Notebook a state-of-the-art model for ASR is presented. A simplified version of this model was implemented to be used in the ASR stage of the pipeline.

## Deep Speech
In 2014 <cite data-cite="undefined"></cite> presented a RNN model called _Deep Speech_ to recognize speech from possible noisy environments. Since the model used CTC, it did non require pre-alignment e.g. through a phonetic transcript. The model was trained on various corpora (_WSJ_, _Switchbboard_, _Fisher_, _Baidu_) containing both conversational and read speech. This data was augmented by artificially adding background noise (_data synthesis_). The model was trained on spectrograms of this audio data. Transcripts for sequences of audio frames were learned using a target alphabet that consisted of 29 characters (`a..z`, `space`, apostrophe and _blank_). Performance was measured with _Label Error Rate_ and _Word Error Rate_. The model architecture was remarkabily simple, consisting of only 5 layers, one of which was an RNN layer:

<figure>
    <img src="../assets/deep_speech_architecture.png" />
    <caption>DeepSpeech architecture (source: <cite data-cite="undefined"></cite>)</caption>
</figure>

### Model layers
The first three layers are dense layers (_fully connected_) and not recurrent. The first layer is a bit special in that it implements a form of convolution by _striding_ the input with a step size of 2. This means that only every second time frame of the original input was taken, but this frame depended on context frames to the left and right. This convolution was done to optimize the training process because it shortens the length of the recurrent layer.

For the first three layers, the _clipped ReLU_ function was used as activation function, which is defined as:

$$
g(z) = \min(\max(0,z), 20)
$$

The fourth layer is recurrent with forward and backward recurrence (_bi-directional_). Hence the units in this layer share weights over time and depend on both the time frames before and after each time step. Note that no LSTM-cells were used in favor of a simpler model that requires less data. _Clipped ReLU_ was again used as activation function for this layer.

The last layer is again dense and non-recurrent. As activation function, softmax was used, yielding a probability for each character of the target alphabet. From the matrix of probabilities with dimensions $(T_x \times 29)$ the loss was calculated by measuring the prediction error with CTC as described before.

### Regularization

To prevent overfitting, dropout with values between 5% and 10% were applied for the first three (non-recurrent) layers. The values of the input signals were centered by subtracting the global mean and then scaled through division by the standard deviation. Additionally, each audio signal was shifted 5ms to the left and right to calculate two additional values per time frame. The probabilities for this time frame were then calculated by averaging over all three values. Finally, an ensemble of several RNN was used at test time.

### Model features and performance

The features used for training were extracted from the audio by calculating spectrograms with a window size of 20ms and a stride of 10ms. The research team did not resort to more sophisticated features like Mel-Spectrograms or MFCC. Despite its simplicity in architecture or feature engineering, the model outperformed previously published systems of that time. This was also possible because a language model (LM) was used to model the probabilities between sequences of letters and words with _n-grams_. This language model fixed error in transcript that were the result of learning plausible renderings of words, that were grammatically incorrect, like the following example taken from the original paper:

| RNN output | Actual transcription |
|---|---|
| bostin | Boston |
| arther n tickets | are there any tickets |

Additionally, performance was improved by introducing a novel approach to parallelize calculations on multiple GPUs. Furthermore, computational effort was reduced through halving the time steps by only using every second time step in the bidirectional layer.

## Model implementation for this project

Because training a state-of-the-art model was not required (and also not feasible) for this project, a simpler model should be trained. A simpler model also allowed for shorter training times and therefore faster feedback cycles. This was crucial for this project as only one GPU was available and the available project time for this stage of the pipeline was very limited. The idea was to find out whether the pipelined approach would still work when using a less capable model in the ASR stage. The ASR-model used in this project is therefore a simplified version of the Deep Speech model as presented in the original paper. Simplification was made in the following aspects:

* no LM was used
* no data synthetization was done, i.e. no audio translation, distortion or superposition of background noise
* the first layer is a simple FC layer, i.e. no striding in the spectrograms was applied to halve the time steps. This also means that no context frames were used to calculate the features for each frame.
* the apostrophe was not part of the target alphabet, thus the target alphabet consisted of 28 characters (`a..z`, `space`, `blank`)
* No ensembling was used
* less training data was available (some hundred hours compared to some thousand for _DeepSpeech_)

The architecture was inspired by code in [this repository](https://github.com/igormq/asr-study). Implementation was done in Python using [Keras](https://keras.io) with [TensorFlow](http://tensorflow.org) backend. The model was trained on both corpora (ReadyLingua and LibriSpeech) using different types of features (MFCC, Mel-Spectrograms, Power-Spectrograms).

### Architecture and optimization

Similar to the original DeepSpeech model, the simplified model consist of 5 layers, whereas the first three layers are fully connected with a dropout of `0.1`.

Optimization was done using the ADAM (Adaptive Moment Estimation) Optimizer <cite data-cite="6174726/8INI335A"></cite>, which is a combination Gradient Descent with Momentum (GDM) and Root Mean Square prop (RMSprop). This Optimizer uses parameters $\beta_1$ and $\beta_2$ as weights for the moving average (from GDM) and the damping (from RMSprop) and has been shown to work well with a wide range of learning problems. Values were set to $b_1=0.9$ and $b_2=0.999$ together with a learning rate of $\alpha=0.1$. For more information see the [Keras API Documentation for Adam](https://keras.io/optimizers/#adam).

Training was done in batches for 20 epochs. After each epoch the CTC loss and the LER was measured, but only the CTC loss was optimized. Decoding was done non-greedily with beam search using a beam width of 100.

#### Input layer
The input layer has shape $(N, T_x, f)$, where $N$ is the batch size, $T_x$ the sequence length and $f$ the number of features. A batch size of $N=5$ was chosen for training. Since the ASR model is trained on speech segments of different lengths, $T_x$ is determined by the longest segment in each batch. Shorter segments are zero-padded to match $T_x$. However, the value of $T_x$ may vary between batches. Like with the PoC, the number of features $f$ (and therefore the number of units in the input layer) depends on the type of features used for training. It was set to default values of $f=13$ for MFCC-features, $f=40$ for Mel-Spectrograms and $f=161$ for power-spectrograms.

Because calculating the features is time-consuming, they were pre-calculated and stored in a [HDF5](https://h5py.org)-file to speed up the training process. According to the [Space-time tradeoff](https://en.wikipedia.org/wiki/Space%E2%80%93time_tradeoff) these files can become quite big.

### Other layers

The other layers correspond more or less to the layers of the _DeepSpeech_ model with the simplifications described above. Execute the following cell to get the Keras summary describing the architecture for a model that can be trained on MFCC-features.


In [ ]:
from util.brnn_util import *

model = deep_speech_model(num_features=13)
model.summary()

## Results

The simplified DeepSpeech model was trained on data from both the _ReadyLingua_ and the _LibriSpeech_ corpus. Since the _ReadyLingua_ corpus contains data in four languages (German, French, Italian and English), separate models had to be trained for each language. The _LibriSpeech_ corpus only contains speech segments in English. Because the model was trained using all three feature types (MFCC, Mel-Spectrograms and Power-Spectrograms), the total number of models was 12 for the _ReadyLingua_ corpus ($4 \cdot 3$) and 3 for the _LibriSpeech_ corpus.

The following paragraphs show the results of the training process for both corpora.

### Training on ReadyLingua data

Because the raw data for the _ReadyLingua_ corpus was not divided into training-, validation- and test-data the data was split into training/dev/test-set using ratios of 80/10/10% for each language.

#### German
The _ReadyLingua_ corpus contains 50 entries in German with a total of  speech segments that are split into train/dev/test-set with the following properties:

|property|train|dev|test|total|
|---|---|---|---|---|
| #speech segments | 1.716 | 215 |215 | 2.146 |
| audio length | 1:21:34 | 0:20:21 | 0:11:35 | 1:43:31 | 


The following plots show the LER loss on training and validation data when trained on MFCC, Mel-Spectrograms and Power-Spectrograms. We can observe that the LER loss in training decreases down to a certain point and then flattens out. The LER loss in validation data is somehow jagged and no clear trend is observable. Because the total length of all speech segments is only approximately 95 minutes (lenght of the training data), it is assumed that this behavior is a direct result of a lack of training data.

<table>
    <tr>
        <td style="text-align:center;">Feature Type</td>
        <td style="text-align:center;">LER-loss (training)</td>
        <td style="text-align:center;">LER-loss (validation)</td>
    </tr>
    <tr>
        <td>MFCC</td>
        <td>
            <img src="../assets/BRNN_rl_de_mfcc_ler_train.png"/>
        </td>
        <td>
            <img src="../assets/BRNN_rl_de_mfcc_ler_val.png" />
        </td>
    </tr>    
    <tr>
        <td>Mel-Spectrograms</td>
        <td>
            <img src="../assets/BRNN_rl_de_mel_ler_train.png" />
        </td>
        <td>
            <img src="../assets/BRNN_rl_de_mel_ler_val.png" />
        </td>
    </tr>    
    <tr>
        <td>Power-Spectrograms</td>
        <td>
            <img src="../assets/BRNN_rl_de_pow_ler_train.png" />
        </td>
        <td>
            <img src="../assets/BRNN_rl_de_pow_ler_val.png" />
        </td>
    </tr>        
</table>

#### English
The _ReadyLingua_ corpus contains 120 entries in English with speech segments that were divided into train/dev/test set with the following properties

|property|train|dev|test|total|
|---|---|---|---|---|
| #speech segments | 4.536 | 567 | 568 | 5.671 |
| audio length | 4:11:17 | 0:22:40 | 0:23:50 | 4:57:48 | 

Although the training data is about three times the size of the training data in German (both in number of speech segments and in audio length), the curves for the LER loss look similar:

<table>
    <tr>
        <td style="text-align:center;">Feature Type</td>
        <td style="text-align:center;">LER-loss (training)</td>
        <td style="text-align:center;">LER-loss (validation)</td>
    </tr>
    <tr>
        <td>MFCC</td>
        <td>
            <img src="../assets/BRNN_rl_en_mfcc_ler_train.png" style="width: 450px;"/>
        </td>
        <td>
            <img src="../assets/BRNN_rl_en_mfcc_ler_val.png" style="width: 450px;"/>
        </td>
    </tr>    
    <tr>
        <td>Mel-Spectrograms</td>
        <td>
            <img src="../assets/BRNN_rl_en_mel_ler_train.png" style="width: 450px;"/>
        </td>
        <td>
            <img src="../assets/BRNN_rl_en_mel_ler_val.png" style="width: 450px;"/>
        </td>
    </tr>    
    <tr>
        <td>Power-Spectrograms</td>
        <td>
            <img src="../assets/BRNN_rl_en_pow_ler_train.png" style="width: 450px;"/>
        </td>
        <td>
            <img src="../assets/BRNN_rl_en_pow_ler_val.png" style="width: 450px;"/>
        </td>
    </tr>        
</table>

### Training on LibriSpeech data

The results of training on the _ReadyLingua_ corpus were expected, because Neural Network usually require vast amounts of data for training which were simply not provided by the corpus. Therefore the _LibriSpeech_ corpus was much more promising because it contains much more data:

|property|train|dev|test|total|
|---|---|---|---|---|
| #speech segments | 4536 | 567 | 568 | 5671 |
| audio length | 1.072:51:03 | 33:53:18 | 34:39:34 | 1.141:23:56 | 

The 5832 entries _LibriSpeech_ corpus are already split into train/dev/test-set because the underlying raw data has already been assigned one of the three sets. The speech segments of those corpus entries are distributed over train/dev/test set with ratios of approximately 94/3/3%.

Training was done on a GPU-cluster whereas one core could be used for this project. Using a GPU gave training a massive speedup compared to the local machine, where the code was developed and which did not have a dedicated GPU. However, because the corpus is so large (more than 1.500h of transcribed audio), the training could still not be finished within the project time, although features were precomputed and Keras/Tensorflow made use of GPU-acceleration. The estimated time for a single epoch when training on MFCC features was roughly 50 hours. With 20 epochs this gives an estimation of 1-2 months training time for each feature type. It is yet to be determined, whether and how this time can further be reduced by optimizations in code, data or by using additional GPU cores.

## Summary

This notebook showed how a RNN for ASR was trained using a simplified version of the architecture proposed in the _DeepSpeech_ paper.

Training on ReadyLingua data did not yield usable results. After training on 20 epochs, the LER was still high and significant progress not observable. As a consequence, the RNN was not able to generate even remotely correct transcripts for unseen samples from the test-set. This might be because the split into training-, validation- and test-data was random and did not consider important properties of the audio signal (like speaker gender, recording quality or acoustic effects). Most probably however it is a result of training on too little training data. It remains to be shown whether good results could be achieved with either more training data or changes in the architecture.

The _LibriSpeech_ corpus was more promising to train the RNN because it contains much more data which was carefully split into training-, validation- and test-set holding data of similar distributions. Howeer, training on _LibriSpeech_ data was not possible within project time (at least not with the proposed setup) because training would have taken much too long with a single GPU core, even if the features were already precomputed.

## References

<div class="cite2c-biblio"></div>